# Groq API Overview - Interacting with LLMs on the Groq Platform

This notebook provides a comprehensive guide to using the Groq API for LLM interactions, covering:
1. Basic chat calls
2. Conversational messaging with message history
3. Function calling capabilities

**Note**: This notebook is optimized for Google Colab but works in any Jupyter environment.

Get your API key from: https://console.groq.com/keys

## Setup and Installation

In [1]:
# Install required packages (run this cell first in Colab)
!pip install groq

print("✅ Packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 7.9 MB/s eta 0:00:00
✅ Packages installed successfully!


In [2]:
import os
import json
from datetime import datetime
import math

# Import Groq after installation
try:
    from groq import Groq
    print("✅ Groq imported successfully!")
except ImportError as e:
    print("❌ Error importing Groq. Make sure you ran the installation cell above.")
    print(f"Error: {e}")

✅ Groq imported successfully!


## API Key Setup

For Google Colab, we'll use Colab's built-in secrets management or direct input:

In [3]:
# Google Colab API Key Setup
import getpass

# Method 1: Use Colab Secrets (Recommended)
# Go to the key icon in the left sidebar and add GROQ_API_KEY
try:
    from google.colab import userdata
    api_key = userdata.get('GROQ_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Direct input (less secure, but works)
    print("Colab secrets not found. Please enter your API key:")
    print("Get your API key from: https://console.groq.com/keys")
    api_key = getpass.getpass("Enter your Groq API key: ")

# Initialize the Groq client
try:
    client = Groq(api_key=api_key)
    print("✅ Groq client initialized successfully!")
except Exception as e:
    print(f"❌ Error initializing Groq client: {e}")
    print("Please check your API key and try again.")

Colab secrets not found. Please enter your API key:
Get your API key from: https://console.groq.com/keys
Enter your Groq API key: ··········
✅ Groq client initialized successfully!


---

## Part 1: Basic Chat Call - Simple Request and Response

Let's start with the simplest way to interact with a Groq LLM - a single request and response.

In [5]:
def basic_chat(user_message, model="llama-3.3-70b-versatile"):
    """
    Simple chat function that sends a message and gets a response

    Args:
        user_message (str): The message to send to the LLM
        model (str): The model to use (default: llama-3.3-70b-versatile)

    Returns:
        str: The LLM's response
    """
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": user_message,
                }
            ],
            model=model,
        )

        return chat_completion.choices[0].message.content

    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
print("=== Basic Chat Example ===")
user_input = "What are some of the best movies of all time"
response = basic_chat(user_input)
print(f"User: {user_input}")
print(f"Assistant: {response}")

=== Basic Chat Example ===
User: What are some of the best movies of all time
Assistant: Here's a list of some of the most iconic and influential movies of all time, across various genres:

**Classics:**

1. **The Godfather (1972)** - a crime drama directed by Francis Ford Coppola
2. **The Shawshank Redemption (1994)** - a drama directed by Frank Darabont
3. **The Wizard of Oz (1939)** - a musical fantasy film directed by Victor Fleming
4. **Casablanca (1942)** - a romantic drama directed by Michael Curtiz
5. **2001: A Space Odyssey (1968)** - a science fiction film directed by Stanley Kubrick

**Sci-Fi and Fantasy:**

1. **Star Wars: Episode IV - A New Hope (1977)** - a space opera directed by George Lucas
2. **The Lord of the Rings: The Fellowship of the Ring (2001)** - a fantasy adventure film directed by Peter Jackson
3. **Blade Runner (1982)** - a science fiction film directed by Ridley Scott
4. **The Matrix (1999)** - a science fiction film directed by the Wachowskis
5. **E.T. th

---

## Part 2: Chat with Message Stack - Conversational AI

For more natural conversations, we need to maintain a message history that gets updated as the conversation progresses.

In [6]:
class GroqConversation:
    """
    A class to manage conversational interactions with Groq LLMs
    """

    def __init__(self, model="llama-3.3-70b-versatile", system_prompt=None):
        self.client = client
        self.model = model
        self.messages = []

        # Add system prompt if provided
        if system_prompt:
            self.messages.append({
                "role": "system",
                "content": system_prompt
            })

    def add_message(self, role, content):
        """Add a message to the conversation history"""
        self.messages.append({
            "role": role,
            "content": content
        })

    def send_message(self, user_message):
        """
        Send a user message and get the assistant's response

        Args:
            user_message (str): The user's message

        Returns:
            str: The assistant's response
        """
        # Add user message to history
        self.add_message("user", user_message)

        try:
            # Get response from Groq
            chat_completion = self.client.chat.completions.create(
                messages=self.messages,
                model=self.model,
                temperature=0.7,
                max_tokens=1000
            )

            # Extract assistant's response
            assistant_response = chat_completion.choices[0].message.content

            # Add assistant response to history
            self.add_message("assistant", assistant_response)

            return assistant_response

        except Exception as e:
            return f"Error: {str(e)}"

    def get_conversation_history(self):
        """Return the full conversation history"""
        return self.messages

    def clear_conversation(self, keep_system=True):
        """Clear conversation history, optionally keeping system prompt"""
        if keep_system and self.messages and self.messages[0]["role"] == "system":
            self.messages = [self.messages[0]]
        else:
            self.messages = []

print("✅ GroqConversation class defined!")

✅ GroqConversation class defined!


In [7]:
# Example: Creating a conversation with a system prompt
print("=== Conversational Chat Example ===")

system_prompt = """You are a helpful AI assistant with expertise in technology and science.
You provide clear, accurate, and engaging explanations. Keep your responses concise but informative."""

conversation = GroqConversation(
    model="llama-3.3-70b-versatile",
    system_prompt=system_prompt
)

# Simulate a multi-turn conversation
conversation_flow = [
    "What is machine learning?",
    "Can you give me a specific example of how it's used in everyday life?",
    "How does that differ from traditional programming?"
]

for i, user_msg in enumerate(conversation_flow, 1):
    print(f"\n--- Turn {i} ---")
    print(f"User: {user_msg}")

    response = conversation.send_message(user_msg)
    print(f"Assistant: {response}")

# Show conversation history
print("\n=== Conversation Summary ===")
history = conversation.get_conversation_history()
print(f"Total messages in history: {len(history)}")
for i, msg in enumerate(history):
    role = msg['role'].upper()
    content_preview = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
    print(f"{i+1}. {role}: {content_preview}")

=== Conversational Chat Example ===

--- Turn 1 ---
User: What is machine learning?
Assistant: Machine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. It enables computers to improve their performance on a task over time, based on experience and data, rather than relying on pre-defined rules.

--- Turn 2 ---
User: Can you give me a specific example of how it's used in everyday life?
Assistant: A common example is virtual assistants like Siri, Google Assistant, or Alexa. They use machine learning to recognize and improve their understanding of voice commands, allowing them to provide more accurate responses and personalized recommendations over time. For instance, if you frequently ask Siri to play a specific music genre, it will learn to prioritize those results in the future.

--- Turn 3 ---
User: How does that differ from traditional programming?
Assistant:

---

## Part 3: Function Calling - Tool Integration

Function calling allows the LLM to use external tools and APIs to provide more accurate and up-to-date information.

In [8]:
# Define some example functions that the LLM can call
def calculate_area_circle(radius):
    """Calculate the area of a circle given its radius"""
    if radius < 0:
        return {"error": "Radius cannot be negative"}
    area = math.pi * radius ** 2
    return {
        "radius": radius,
        "area": round(area, 2),
        "formula": "π × r²"
    }

def get_current_time():
    """Get the current date and time"""
    now = datetime.now()
    return {
        "current_time": now.strftime("%Y-%m-%d %H:%M:%S"),
        "timezone": "Local",
        "day_of_week": now.strftime("%A"),
        "month": now.strftime("%B")
    }

def calculate_compound_interest(principal, rate, time, compound_frequency=1):
    """
    Calculate compound interest

    Args:
        principal: Initial amount
        rate: Annual interest rate (as decimal, e.g., 0.05 for 5%)
        time: Time in years
        compound_frequency: How many times interest is compounded per year
    """
    amount = principal * (1 + rate/compound_frequency) ** (compound_frequency * time)
    interest = amount - principal

    return {
        "principal": principal,
        "rate": rate * 100,  # Convert to percentage
        "time_years": time,
        "compound_frequency": compound_frequency,
        "final_amount": round(amount, 2),
        "interest_earned": round(interest, 2)
    }

# Function registry - maps function names to actual functions
AVAILABLE_FUNCTIONS = {
    "calculate_area_circle": calculate_area_circle,
    "get_current_time": get_current_time,
    "calculate_compound_interest": calculate_compound_interest,
}

print("✅ Functions defined successfully!")

✅ Functions defined successfully!


In [9]:
# Function definitions for the LLM (in the format Groq expects)
function_definitions = [
    {
        "type": "function",
        "function": {
            "name": "calculate_area_circle",
            "description": "Calculate the area of a circle given its radius",
            "parameters": {
                "type": "object",
                "properties": {
                    "radius": {
                        "type": "number",
                        "description": "The radius of the circle"
                    }
                },
                "required": ["radius"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current date and time",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_compound_interest",
            "description": "Calculate compound interest for an investment",
            "parameters": {
                "type": "object",
                "properties": {
                    "principal": {
                        "type": "number",
                        "description": "Initial investment amount"
                    },
                    "rate": {
                        "type": "number",
                        "description": "Annual interest rate as a decimal (e.g., 0.05 for 5%)"
                    },
                    "time": {
                        "type": "number",
                        "description": "Investment time in years"
                    },
                    "compound_frequency": {
                        "type": "number",
                        "description": "Number of times interest is compounded per year (default: 1)",
                        "default": 1
                    }
                },
                "required": ["principal", "rate", "time"]
            }
        }
    }
]

print("✅ Function definitions created!")

✅ Function definitions created!


In [12]:
class GroqFunctionCalling:
    """
    A class to handle Groq conversations with function calling capabilities
    """

    def __init__(self, model="llama-3.3-70b-versatile"):
        self.client = client
        self.model = model
        self.messages = []
        self.functions = function_definitions
        self.available_functions = AVAILABLE_FUNCTIONS

    def send_message_with_functions(self, user_message):
        """
        Send a message that can trigger function calls
        """
        # Add user message
        self.messages.append({
            "role": "user",
            "content": user_message
        })

        try:
            # Make the initial request with function definitions
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                tools=self.functions,
                tool_choice="auto"  # Let the model decide when to use functions
            )

            response_message = response.choices[0].message

            # Check if the model wants to call a function
            if response_message.tool_calls:
                # Add the assistant's response to messages
                self.messages.append({
                    "role": "assistant",
                    "content": response_message.content,
                    "tool_calls": response_message.tool_calls
                })

                # Process each function call
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name

                    # Safely parse function arguments
                    try:
                        function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}
                        # Handle case where function_args might be None or empty
                        if function_args is None:
                            function_args = {}
                    except (json.JSONDecodeError, TypeError):
                        function_args = {}

                    print(f"🔧 Calling function: {function_name}")
                    print(f"📝 Arguments: {function_args}")

                    # Call the actual function
                    if function_name in self.available_functions:
                        function_result = self.available_functions[function_name](**function_args)

                        # Add function result to messages
                        self.messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "name": function_name,
                            "content": json.dumps(function_result)
                        })

                # Get the final response after function calls
                final_response = self.client.chat.completions.create(
                    model=self.model,
                    messages=self.messages,
                )

                final_message = final_response.choices[0].message.content
                self.messages.append({
                    "role": "assistant",
                    "content": final_message
                })

                return final_message

            else:
                # No function call needed, just return the response
                self.messages.append({
                    "role": "assistant",
                    "content": response_message.content
                })
                return response_message.content

        except Exception as e:
            return f"Error: {str(e)}"

print("✅ GroqFunctionCalling class defined!")

✅ GroqFunctionCalling class defined!


In [13]:
# Example usage
print("=== Function Calling Examples ===")

function_chat = GroqFunctionCalling()

# Test cases that should trigger different functions
test_queries = [
    "What time is it right now?",
    "Calculate the area of a circle with radius 5",
    "If I invest $1000 at 5% annual interest for 3 years, how much will I have?",
    "Hello, how are you today?"  # This shouldn't trigger any functions
]

for i, query in enumerate(test_queries, 1):
    print(f"\n--- Example {i} ---")
    print(f"User: {query}")

    response = function_chat.send_message_with_functions(query)
    print(f"Assistant: {response}")
    print("-" * 50)

=== Function Calling Examples ===

--- Example 1 ---
User: What time is it right now?
🔧 Calling function: get_current_time
📝 Arguments: {}
Assistant: The current time is 5:41:07 PM.
--------------------------------------------------

--- Example 2 ---
User: Calculate the area of a circle with radius 5
🔧 Calling function: calculate_area_circle
📝 Arguments: {'radius': 5}
Assistant: The area of a circle with radius 5 is 78.54 square units.
--------------------------------------------------

--- Example 3 ---
User: If I invest $1000 at 5% annual interest for 3 years, how much will I have?
🔧 Calling function: calculate_compound_interest
📝 Arguments: {'principal': 1000, 'rate': 0.05, 'time': 3}
Assistant: After 3 years, you will have $1157.63, earning $157.63 in interest.
--------------------------------------------------

--- Example 4 ---
User: Hello, how are you today?
Assistant: I'm just a language model, so I don't have feelings or emotions like humans do, but I'm here to help answer yo